### Create permutations of possible parameters

In [ ]:
from itertools import product
import numpy as np

input_nb_path = './minibatch-kmeans.ipynb'

parameter_dict = {
'surface_dimensions' : [[9,9]],
'time_constant' : [50000, 100000],
'dropout_probability' : [0, 0.5, 0.9],
#'dropout_probability' : [0],
'refractory_period' : [0, 50, 500],
#'refractory_period' : [0],
'n_of_centers' : [100, 500, 1000],
}

value_permutations = list(product(*list(parameter_dict.values())))
print('First set of ' + str(len(value_permutations)) + ' parameter combinations: '+ str(dict(zip(parameter_dict.keys(), value_permutations[0]))))

short_keys = ['_'+k[:4]+'_' for k in list(parameter_dict.keys())] # will be used for nb filenames

### Execute notebooks in parallel

In [ ]:
import multiprocessing
import os, glob
import time
from functools import partial
import papermill as pm
from joblib import Parallel, delayed, parallel_backend

def process_notebook(params, short_keys, parameter_dict, input_nb_path):
    string = ''
    for index, key in enumerate(short_keys):
        string += key + str(params[index])
    string += '.ipynb'

    param_dict = dict(zip(parameter_dict.keys(), params))
    param_dict['file_name'] = string
    
    output = './milled_nbs/' + string
    pm.execute_notebook(
       input_nb_path,
       output,
       parameters = param_dict
    )

if __name__ == '__main__':
    start_time = time.time()
    with multiprocessing.Pool() as pool:
        pool.map(partial(process_notebook, short_keys=short_keys, parameter_dict=parameter_dict, input_nb_path=input_nb_path), value_permutations)
    print("--- %s seconds ---" % (time.time() - start_time))
    for filename in glob.glob("./milled_nbs/_surf*"):
        os.remove(filename) 